# Sw00pGenerator3000 - SG3K - Flysight 

<br/>

The SwoopGenerator3000 (SG3K) is an advanced GPS analysis tool specifically developed to assist swoopers in assessing and enhancing their performance. Whether you're a novice embarking on your swooping journey or a seasoned competitor with extensive experience, the SG3K will expedite your learning process, enabling you to make remarkable progress while minimizing the number of jumps required.


![flysight](img/product.jpg)


**FlySight is not your mother's GPS.**<br/>
<br/>
FlySight was designed from the ground up for wingsuit pilots and does one truly revolutionary thing...

**FlySight provides real-time audible indication of glide ratio, horizontal or vertical speed.**<br/>
<br/>
If you've used other GPS receivers, you know the drill. When you land, you review data from the jump. If you're lucky, or if you've planned a very simple jump, you might remember what you were doing when your glide ratio maxed out. On the next jump, you try to do it again.

**FlySight speeds the learning process.**<br/>
<br/>
When you change your body position, you instantly hear a change in the tone. Your brain easily connects this feedback with your precise body position, making it astonishingly easy to remember what worked and what didn't.


https://github.com/flysight/flysight

<br/>

---
The variables in the dataset generated by the flysight have the following meaning:

| **Name**   | **Unit**  | **Description**                               |
|:-----------|:----------|:----------------------------------------------|
| time       | datetime  | Time in ISO8601 format                        |
| lat        | deg       | Latitude                                      |
| lon        | deg       | Longitude                                     |
| hMSL       | m         | Height above mean sea level                   |
| velN       | m/s       | NED north velocity                            |
| velE       | m/s       | NED east velocity                             |
| velD       | m/s       | NED down velocity                             |
| hAcc       | m         | Horizontal Accuracy Estimate                  |
| vAcc       | m         | Vertical Accuracy Estimate                    |
| sAcc       | m/s       | Speed Accuracy Estimate                       |
| heading    | deg       | Heading of motion 2-D                         |
| cAcc       | deg       | Course / Heading Accuracy Estimate            |
| gpsFix     | -         | GPSfix Type (3 = 3D)                          |
| numSV      | -         | # of satellites following.                    |

https://content.u-blox.com/sites/default/files/products/documents/u-blox6_ReceiverDescrProtSpec_%28GPS.G6-SW-10018%29_Public.pdf

In [25]:
%matplotlib inline
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import peakutils as pu
from plotly.subplots import make_subplots

import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from math import sqrt, degrees, atan, pi
import geopy.distance
import pyproj
import utm
import textwrap

from jupyter_dash import JupyterDash 
from dash import Dash, dcc, html, Input, Output, callback

import logging
logging.basicConfig(level=logging.INFO)

themes = ["plotly_white", "plotly_dark", "ggplot2", "simple_white"]
pio.templates.default = themes[0]

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

token = "pk.eyJ1IjoiYWlieWJydW0iLCJhIjoiY2xhdmRldnhyMDRkNTNybWg0NTl6eHgwZSJ9.phqQCg9UWIXx5wFkc_i0kg"

In [26]:
raw_df = pd.read_csv('../data/raw_own/23-04-09/J1.CSV', skiprows=[1])
raw_df.head()

,time,lat,lon,hMSL,velN,velE,velD,hAcc,vAcc,sAcc,heading,cAcc,gpsFix,numSV
0,2023-04-09T07:28:07.40Z,51.024551,5.520155,1348.562,-16.88,5.83,21.29,4307.536,5007.471,224.70,160.95330,7.66592,3,4
1,2023-04-09T07:28:07.60Z,51.023893,5.520543,1260.528,-18.41,6.39,23.06,1904.139,2209.380,96.17,160.86574,3.90365,3,4
2,2023-04-09T07:28:07.80Z,51.016739,5.524668,283.871,-1.86,0.36,2.70,1174.856,1382.874,39.04,160.86574,4.71886,3,4
3,2023-04-09T07:28:08.00Z,51.015957,5.525116,174.914,0.29,-0.04,-0.66,803.097,952.791,23.55,160.86574,5.14861,3,4
4,2023-04-09T07:28:08.20Z,51.015678,5.525263,134.521,0.00,-0.01,-0.04,607.534,727.155,14.30,160.86574,5.39289,3,5


In [27]:
# Dataset helpers

def meters_to_feet(meters):
    return meters * 3.280839895

def feet_to_meters(feet):
    return feet / 3.280839895

def meter_per_second_to_miles_per_hour(meter_per_second):
    return meter_per_second * 2.236936

def meter_per_second_to_kilometers_per_hour(meter_per_second):
    return meter_per_second * 3.6

def calc_horizontal_speed(n, e):
    return sqrt((n**2) + (e**2))

def calc_dive_angle(v_speed, h_speed):
    try:
        return degrees(atan(v_speed/h_speed))
    except ZeroDivisionError:
        return 0

def calc_distance_geo(metric, lat1, lat2, lon1, lon2):
    coordinates_1 = (lat1, lon1)
    coordinates_2 = (lat2, lon2)
    if metric == 'ft':
        return geopy.distance.geodesic(coordinates_1, coordinates_2).miles * 5280
    elif metric == 'm':
        return geopy.distance.geodesic(coordinates_1, coordinates_2).kilometers * 1000

def to_utm(lat, lon):
    zone = utm.from_latlon(lat[0], lon[0])[2]
    p = pyproj.Proj(proj='utm', zone=zone, ellps='WGS84')
    return p(lon, lat)

def calc_axis_distance(metric, lat, lon):
    x, y = to_utm(lat, lon)
    if metric == 'ft':
        return [meters_to_feet(x_cor) for x_cor in x], [meters_to_feet(y_cor) for y_cor in y]
    elif metric == 'm':
        return x, y

In [28]:
# Creating dataset

class Dataset:
    def __init__(self, filename, df, user_id=None):
        self.filename = filename
        self.df = df
        self.user_id = user_id

    def get_total_seconds(self):
        datetimes = [pd.to_datetime(d) for d in self.df.time]
        l = []
        for i, d in enumerate(datetimes):
            duration = datetimes[i] - datetimes[0]
            l.append(duration.total_seconds())
        return l

    def get_fixed_elevation(self, elevation):
        ground_elevation = meters_to_feet(self.df.hMSL.iloc[-1])
        return [meters_to_feet(self.df.hMSL[i]) - ground_elevation - elevation for i in
                range(0, len(self.df.hMSL))]

    def get_vertical_speed(self, metric):
        if metric == 'mph':
            return [meter_per_second_to_miles_per_hour(meter) for meter in self.df.velD]
        elif metric == 'km/u':
            return [meter_per_second_to_kilometers_per_hour(meter) for meter in self.df.velD]
        else:
            raise ValueError('Invalid metric')

    def get_horizontal_speed(self, metric):
        speed = [calc_horizontal_speed(self.df.velN[i], self.df.velE[i]) for i in range(0, len(self.df))]
        if metric == 'mph':
            return [meter_per_second_to_miles_per_hour(s) for s in speed]
        elif metric == 'km/u':
            return [meter_per_second_to_kilometers_per_hour(s) for s in speed]
        else:
            raise ValueError('Invalid metric')

    def get_dive_angle(self, v_speed, h_speed):
        return [calc_dive_angle(v_speed[i], h_speed[i]) for i in range(0, len(self.df))]

    def get_horizontal_distance(self, metric):
        lis, dis = [0], 0.0
        for i in range(0, len(self.df) - 1):
            dis += calc_distance_geo(metric, self.df.lat[i], self.df.lat[i + 1], self.df.lon[i], self.df.lon[i + 1])
            lis.append(dis)
        return lis

    def get_axis_distance(self, metric, axis):
        x, y = calc_axis_distance(metric, self.df.lat, self.df.lon)
        axis_value = x if axis == 'x' else y
        lis, dis = [0], 0.0
        for i in range(0, len(axis_value) - 1):
            dis += axis_value[i + 1] - axis_value[i]
            lis.append(dis)
        return lis
    
    def calculate_rate_of_turn(self):
        time_interval = 0.2
        rate_of_turns_deg_per_sec = [0]
        for i in range(1, len(self.df.heading)):
            delta_heading_deg = abs(self.df.heading[i] - self.df.heading[i - 1])
            rate_of_turn_deg_per_sec = delta_heading_deg / time_interval
            rate_of_turns_deg_per_sec.append(rate_of_turn_deg_per_sec)
        return rate_of_turns_deg_per_sec
    
    def calculate_degrees_of_rotation(self):
        degrees_of_rotation = [0]
        for i in range(1, len(self.df.heading)):
            delta_heading_deg = abs(self.df.heading[i] - self.df.heading[i - 1])
            degrees_of_rotation.append(delta_heading_deg)
        return degrees_of_rotation

    def create(self):
        df = pd.DataFrame({
            'timestamp': pd.to_datetime(self.df.time),
            'time': np.array(self.get_total_seconds()),
            'lat': self.df.lat,
            'lon': self.df.lon,
            'elevation': self.get_fixed_elevation(0),
            'horz_distance_ft': self.get_horizontal_distance('ft'),
            'horz_distance_m': self.get_horizontal_distance('m'),
            'x_axis_distance_ft': self.get_axis_distance('ft', 'x'),
            'x_axis_distance_m': self.get_axis_distance('m', 'x'),
            'y_axis_distance_ft': self.get_axis_distance('ft', 'y'),
            'y_axis_distance_m': self.get_axis_distance('m', 'y'),
            'vert_speed_mph': self.get_vertical_speed('mph'),
            'horz_speed_mph': self.get_horizontal_speed('mph'),
            'vert_speed_km/u': self.get_vertical_speed('km/u'),
            'horz_speed_km/u': self.get_horizontal_speed('km/u'),
            'heading': self.df.heading,
            'dive_angle': self.get_dive_angle(self.get_vertical_speed('mph'), self.get_horizontal_speed('mph')),
            'name': self.get_name(),
            'user_id': self.user_id})
        #df.set_index('timestamp', inplace=True)
        return df

    def get_name(self):
        return pd.to_datetime(self.df.time[0]).strftime("D%m-%d-%YT%H%M") + self.filename

    def save(self, path):
        dataframe = self.create()
        dataframe.to_csv(os.path.join(path, self.get_name() + '.csv'), index=False)

    def __str__(self):
        return f'{self.get_name()}'

## Dataset

The "Dataset" class creates a fresh dataset from the Flysight device. This dataset can be stored in InfluxDB to preserve the jump information. The variables in the newly generated dataset carry the following significance:

| **Name**           | **Description**                               |
|:-------------------|:----------------------------------------------|
| timestamp          | year-month-day hour:minute:second:millisecond |
| time_sec           | Time (s)                                      |
| lat                | Latitude                                      |
| lon                | Longitude                                     |
| elevation          | Elevation                                     |
| horz_distance_ft   | Horizontal Distance feet                      |
| horz_distance_m    | Horizontal Distance meter                     |
| x_axis_distance_ft | X Axis Distance feet                          |
| x_axis_distance_m  | X Axis Distance meter                         |
| y_axis_distance_ft | Y Axis Distance feet                          |
| y_axis_distance_m  | Y Axis Distance meter                         |
| vert_speed_mph     | Vertical speed miles per hour                 |
| horz_speed_mph     | Horizontal speed miles per hour               |
| vert_speed_km/u    | Vertical speed kilometers per hour            |
| horz_speed_km/u    | Horizontal speed kilometers per hour          |
| heading            | Heading                                       |
| dive_angle         | Dive angle                                    |
| turn_rate          | Rate of turn                                  |
| name               | Name of dataset                               |
| user_id            | User id that generated the data               |

In [196]:
#raw = pd.read_csv('../data/raw_own/23-04-08/J1.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-08/J2.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-08/J3.CSV', skiprows=[1])

#raw = pd.read_csv('../data/raw_own/23-04-09/J1.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-09/J2.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-09/J3.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-09/J4.CSV', skiprows=[1])

#raw = pd.read_csv('../data/raw_own/23-04-15/J1.CSV', skiprows=[1])
#raw = pd.read_csv('../data/raw_own/23-04-15/J2.CSV', skiprows=[1])

# Bad exit calc
#raw = pd.read_csv('../data/raw_own/23-04-15/J3.CSV', skiprows=[1])

raw = pd.read_csv('../data/raw_magicmike/J1.CSV', skiprows=[1])
dataset = Dataset("v1", raw).create()
dataset.head()

,timestamp,time,lat,lon,elevation,horz_distance_ft,horz_distance_m,x_axis_distance_ft,x_axis_distance_m,y_axis_distance_ft,y_axis_distance_m,vert_speed_mph,horz_speed_mph,vert_speed_km/u,horz_speed_km/u,heading,dive_angle,name,user_id
0,2022-06-22 11:45:46.800000+00:00,0.0,56.183217,9.040273,-207.821522,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.714923,5.809879,-17.244,9.350095,340.00856,-61.532429,D06-22-2022T1145v1,None
1,2022-06-22 11:45:47+00:00,0.2,56.183230,9.040298,-205.377297,7.093672,2.162151,5.129113,1.563354,4.896135,1.492342,-4.115962,4.182173,-6.624,6.730556,344.61396,-44.542847,D06-22-2022T1145v1,None
2,2022-06-22 11:45:47.200000+00:00,0.4,56.183236,9.040297,-213.503937,9.145866,2.787660,4.964999,1.513332,6.940933,2.115596,-2.818539,4.393629,-4.536,7.070862,335.28268,-32.680413,D06-22-2022T1145v1,None
3,2022-06-22 11:45:47.400000+00:00,0.6,56.183298,9.040240,-193.500656,34.751788,10.592345,-6.635981,-2.022647,29.756633,9.069822,-2.348783,2.550499,-3.780,4.104632,335.28268,-42.642310,D06-22-2022T1145v1,None
4,2022-06-22 11:45:47.600000+00:00,0.8,56.183349,9.040209,-166.400919,54.445333,16.594937,-12.919259,-3.937790,48.412624,14.756168,-1.923765,0.552482,-3.096,0.889134,335.28268,-73.976563,D06-22-2022T1145v1,None


In [197]:
# Helper functions

'''
Set starting point of dataframe
'''

def cumulative_sum(df, fase='1', between=0.0):
    lis, diff = [0.0], 0.0
    if fase == '2':
        lis[0] = round(between, 6)
        diff = lis[0]
    for i in range(len(df) - 1):
        diff += df[i+1] - df[i]
        lis.append(round(diff, 6))
    return lis


def shift_df(dataframe, key, col):
    if key < 0:
        raise ValueError("key must be positive")
    if key >= len(dataframe):
        raise ValueError("key must be less than the length of dataframe")
    first = dataframe.iloc[:key+1][col][::-1].reset_index(drop=True)
    second = dataframe.iloc[key+1:][col].reset_index(drop=True)
    between = second.iloc[0] - first.iloc[0]
    return np.concatenate((cumulative_sum(first, fase='1')[::-1], cumulative_sum(second, fase='2', between=between)))


def set_start_point(df, key):
    df2 = df.copy()
    df2['time'] = shift_df(df2, key, 'time')
    df2['horz_distance_m'] = shift_df(df2, key, 'horz_distance_m')
    df2['horz_distance_ft'] = shift_df(df2, key, 'horz_distance_ft')
    df2['x_axis_distance_m'] = shift_df(df2, key, 'x_axis_distance_m')
    df2['x_axis_distance_ft'] = shift_df(df2, key, 'x_axis_distance_ft')
    df2['y_axis_distance_m'] = shift_df(df2, key, 'y_axis_distance_m')
    df2['y_axis_distance_ft'] = shift_df(df2, key, 'y_axis_distance_ft')
    return df2

## ---- ##

def validate_metric(speed_metric, distance_metric):
    if speed_metric != 'km/u' and speed_metric != 'mph':
        raise ValueError("Wrong speed metric")
    if distance_metric != 'm' and distance_metric != 'ft':
        raise ValueError("Wrong distance metric")
        
def find_peaks_lows(metric, thres_peaks, min_dist_peaks, thres_lows, min_dist_lows):
    peaks = pu.indexes(metric, thres=thres_peaks, min_dist=min_dist_peaks)
    lows = pu.indexes(-metric, thres=thres_lows, min_dist=min_dist_lows)
    return {'peaks': peaks, 'lows': lows}

def get_y_axis_settings(df, speed_metric='km/u', distance_metric='m'):
    validate_metric(speed_metric, distance_metric)
    return {
        'Elevation':
        {
            'col': df.elevation, 
            'color': '#636EFA', 
            'metric': "ft",
            'hovertemplate': 'Elevation: %{y:.2f} ft <extra></extra>'
        },
        'Horizontal speed': 
        {
            'col': df['horz_speed_km/u'] if speed_metric == 'km/u' else df['horz_speed_mph'], 
            'color': '#FF0B0B', 
            'metric': speed_metric,
            'hovertemplate': 'Horz speed: %{y:.2f} ' + speed_metric + '<extra></extra>'
        },
        'Vertical speed':   
        {
            'col': df['vert_speed_km/u'] if speed_metric == 'km/u' else df['vert_speed_mph'],
            'color': '#00CC96', 
            'metric': speed_metric,
            'hovertemplate': 'Vert speed: %{y:.2f} ' + speed_metric + '<extra></extra>'
        },
        'Dive angle':       
        {
            'col': df.dive_angle,
            'color': '#AB63FA', 
            'metric': 'deg',
            'hovertemplate': 'Dive angle: %{y:.2f}° <extra></extra>'
        },
        'Heading':       
        {
            'col': df.heading,
            'color': '#032B43', 
            'metric': 'deg',
            'hovertemplate': 'Heading: %{y:.2f}° <extra></extra>'
        },
        'Distance':   
        {
            'col': df['y_axis_distance_m'] if distance_metric == "m" else df['y_axis_distance_ft'],
            'color': '#636EFA', 
            'metric': distance_metric,
            'hovertemplate': 'yaxis distance: %{y:.2f} ' + distance_metric + '<extra></extra>'
        },
    }

def get_x_axis_settings(df, speed_metric='km/u', distance_metric='m'):
    validate_metric(speed_metric, distance_metric)
    return {
        'Time': 
        {
            'col': df.time,
            'metric': 's'
        },
        'Horizontal Distance': 
        {
            'col': df['horz_distance_m'] if distance_metric == "m" else df['horz_distance_ft'],
            'color': '#636EFA', 
            'metric': distance_metric,
            'hovertemplate': 'Horizontal Distance: %{y:.2f} ' + distance_metric + '<extra></extra>'
        },
        'Distance': 
        {
            'col': df['x_axis_distance_m'] if distance_metric == "m" else df['x_axis_distance_ft'],
            'metric': distance_metric
        }
    }

def get_metrics(df, idke, speed_metric):
    y_axis = get_yaxis_settings(df, speed_metric)
    return {
        'y_axis':
        {
            'values': [y_axis[col]['col'].loc[y_axis[col]['col'].index == idke].values[0] for col in y_axis.keys()], 
            'metric': [y_axis[col]['metric'] for col in y_axis.keys()],
        },
        
    }

def empty_layout(text):
    return {
        'layout': go.Layout(
            xaxis={"visible": True},
            yaxis={"visible": True},
            annotations=[
                {
                    "text": text,
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 28
                    }
                }
            ]
        )
    }

### Exit

The provided information offers a highly accurate indication of the exit altitude. When exiting the plane during a hop-n-pop while it's still ascending, you may experience an upward force, but the GPS can precisely determine the exit location by analyzing the downward acceleration.

In [198]:
class Exit:
    def __init__(self, df):
        self.df = df
        self.exit_df = self.df.iloc[self.get_exit():].reset_index(drop=True)

    def get_exit_elevation(self):
        peaks = pu.indexes(self.df.elevation, thres=0.9, min_dist=1)
        offset = peaks[-1] - 30
        jmp_df = self.df.iloc[offset:].reset_index()
        return {'plane_on_altitude': jmp_df, 'peaks': peaks}

    def get_exit_horz_speed(self):
        jmp_df = self.get_exit_elevation()['plane_on_altitude']
        horz_speed_peaks = pu.indexes(jmp_df.horz_speed_mph, thres=0.8, min_dist=2)
        horz_speed_lows = pu.indexes(-jmp_df.horz_speed_mph, thres=0.4, min_dist=2)
        peak = [p for p in horz_speed_peaks if p < horz_speed_lows[0]]
        return {'exit': peak[-1], 'peaks': horz_speed_peaks, 'lows': horz_speed_lows}

    def get_exit_vert_speed(self):
        jmp_df = self.get_exit_elevation()['plane_on_altitude']
        vert_speed_peaks = pu.indexes(jmp_df.vert_speed_mph, thres=0.4, min_dist=2)
        vert_speed_lows = pu.indexes(-jmp_df.vert_speed_mph, thres=0.8, min_dist=2)
        drop = [l for l in vert_speed_lows if l < vert_speed_peaks[0]]
        return {'exit': drop[-1], 'peaks': vert_speed_peaks, 'lows': vert_speed_lows}
    
    @staticmethod
    def closest_value(input_list, input_value):
        difference = lambda x: abs(x - input_value)
        res = min(input_list, key=difference)
        return res

    def get_exit(self):
        jmp_df = self.get_exit_elevation()['plane_on_altitude']
        mean = np.mean([jmp_df.time[self.get_exit_vert_speed()['exit']], 
                        jmp_df.time[self.get_exit_horz_speed()['exit']]])
        return self.df.index[self.df['time'] == self.closest_value(self.df.time, mean)][0]
    
    def plt_exit_debug(self, speed_metric, distance_metric):
        fig = make_subplots(rows=2, cols=2, specs=[[{"colspan": 2}, None],[{}, {}]], vertical_spacing=0.1,
                            shared_xaxes=True, shared_yaxes=False, subplot_titles=("Elevation","Horizontal speed", "Vertical speed"))
        
        total_x_axis = get_x_axis_settings(self.df, distance_metric=distance_metric)
        total_y_axis = get_y_axis_settings(self.df, speed_metric=speed_metric)
        
        elevation_peaks = self.get_exit_elevation()['peaks']
        fig.add_trace(
            go.Scatter(
                x=total_x_axis['Horizontal Distance']['col'], 
                y=total_y_axis['Elevation']['col'],
                name='Elevation', 
                line=dict(color=total_y_axis['Elevation']['color'], width=1.2),
                hovertemplate=total_y_axis['Elevation']['hovertemplate'], 
                showlegend=False
            ),
            row=1, col=1
        )
        
        fig.add_vline(x=total_x_axis['Horizontal Distance']['col'][elevation_peaks[-1]], line_width=1.2, row=1, col=1)
        fig.add_trace(
            go.Scatter(
                mode='markers',
                x=total_x_axis['Horizontal Distance']['col'][elevation_peaks], 
                y=total_y_axis['Elevation']['col'][elevation_peaks],
                marker=dict(size=7, color='Green'),
                showlegend=False
            ),
            row=1, col=1
        )
        
        fig.update_xaxes(title_text='Horizontal distance (' + total_x_axis['Horizontal Distance']['metric'] + ')', row=1, col=1)
        fig.update_yaxes(title_text='Elevation (' + total_y_axis['Elevation']['metric'] + ')', row=1, col=1)
        
        jmp_df = self.get_exit_elevation()['plane_on_altitude']
        
        x_axis = get_x_axis_settings(jmp_df, distance_metric=distance_metric)
        y_axis = get_y_axis_settings(jmp_df, speed_metric=speed_metric)
        
        horz_speed = self.get_exit_horz_speed()
        fig.add_trace(
            go.Scatter(
                x=x_axis['Horizontal Distance']['col'], 
                y=y_axis['Horizontal speed']['col'],
                name='Horizontal speed', 
                line=dict(color=y_axis['Horizontal speed']['color'], width=1.2),
                hovertemplate=y_axis['Horizontal speed']['hovertemplate'], 
                showlegend=False
            ),
            row=2, col=1
        )
        
        fig.add_vline(x=x_axis['Horizontal Distance']['col'][horz_speed['exit']], line_width=1.2, row=2, col=1)
        fig.add_trace(
            go.Scatter(
                mode='markers', 
                x=x_axis['Horizontal Distance']['col'][horz_speed['peaks']], 
                y=y_axis['Horizontal speed']['col'][horz_speed['peaks']],
                marker=dict(size=7, color='Green'),
                showlegend=False
            ),
            row=2, col=1
        )
        fig.add_trace(
            go.Scatter(
                mode='markers', 
                x=x_axis['Horizontal Distance']['col'][horz_speed['lows']], 
                y=y_axis['Horizontal speed']['col'][horz_speed['lows']],
                marker=dict(size=7, color='Red'),
                showlegend=False
            ),
            row=2, col=1
        )
        
        fig.update_xaxes(title_text='Horizontal distance (' + x_axis['Horizontal Distance']['metric'] + ')', row=2, col=1)
        fig.update_yaxes(title_text='Horizontal speed (' + y_axis['Horizontal speed']['metric'] + ')', row=2, col=1)
        
        vert_speed = self.get_exit_vert_speed()
        fig.add_trace(
            go.Scatter(
                x=x_axis['Horizontal Distance']['col'], 
                y=y_axis['Vertical speed']['col'],
                name='Vertical speed', 
                line=dict(color=y_axis['Vertical speed']['color'], width=1.2),
                hovertemplate=y_axis['Vertical speed']['hovertemplate'], 
                showlegend=False
            ),
            row=2, col=2
        )
        
        fig.add_vline(x=x_axis['Horizontal Distance']['col'][vert_speed['exit']], line_width=1.2, row=2, col=2)
        fig.add_trace(
            go.Scatter(
                mode='markers', 
                x=x_axis['Horizontal Distance']['col'][vert_speed['peaks']], 
                y=y_axis['Vertical speed']['col'][vert_speed['peaks']],
                marker=dict(size=7, color='Green'),
                showlegend=False
            ),
            row=2, col=2
        )
        fig.add_trace(
            go.Scatter(
                mode='markers', 
                x=x_axis['Horizontal Distance']['col'][vert_speed['lows']], 
                y=y_axis['Vertical speed']['col'][vert_speed['lows']],
                marker=dict(size=7, color='Red'),
                showlegend=False
            ),
            row=2, col=2
        )
        
        fig.update_xaxes(title_text='Horizontal distance (' + x_axis['Horizontal Distance']['metric'] + ')', row=2, col=2)
        fig.update_yaxes(title_text='Vertical speed (' + y_axis['Vertical speed']['metric'] + ')', row=2, col=2)

        fig.update_layout(hovermode='x unified', height=1000, showlegend=False, title_text="Exit - Debug")
        return fig
    
    def plt_overview(self, selected, speed_metric, distance_metric):
        if len(selected) != 0:
            x_axis = get_x_axis_settings(self.exit_df, distance_metric=distance_metric)
            y_axis = get_y_axis_settings(self.exit_df, speed_metric=speed_metric)

            plotly_data = []
            layout_kwargs = {'xaxis': {'domain': [0.06 * (len(selected) - 1), 1],
                                       'title': 'Horizontal distance (' + distance_metric + ')'}}

            for i, s in enumerate(selected):
                axis_name = 'yaxis' + str(i + 1) * (i > 0)
                yaxis = 'y' + str(i + 1) * (i > 0)
                plotly_data.append(go.Scatter(
                    x=x_axis['Horizontal Distance']['col'],
                    y=y_axis[s]['col'],
                    name=s,
                    mode='lines',
                    line=dict(color=y_axis[s]['color'], width=1.2),
                    showlegend=False,
                    hovertemplate=y_axis[s]['hovertemplate'],
                ),
                )
                layout_kwargs[axis_name] = {
                    'position': i * 0.06, 'side': 'left', 'title': s + ' (' + y_axis[s]['metric'] + ')',
                    'titlefont': {'size': 10, 'color': y_axis[s]['color']}, 'title_standoff': 0,
                    'anchor': 'free', 'tickfont': {'size': 10, 'color': y_axis[s]['color']}, 'showgrid': False,
                }

                plotly_data[i]['yaxis'] = yaxis
                if i > 0:
                    layout_kwargs[axis_name]['overlaying'] = 'y'

            fig = go.Figure(data=plotly_data, layout=go.Layout(**layout_kwargs))
            fig.update_layout(
                hovermode="x unified",
                title_text="Jump"
            )
            return fig
        else:
            return go.Figure(empty_layout("Please select Y-axis"))

    def get_exit_df(self):
        return self.exit_df


In [199]:
class Jump(Exit):
    def __init__(self, name="empty", df=pd.DataFrame()):
        self.name = name
        if not df.empty:
            self.df = df
            super().__init__(self.df)
        
    def get_df(self):
        return self.df

    def get_name(self):
        return self.name

    def set_name(self, name):
        self.name = name

    def __str__(self):
        return f'{self.name}'
    
jump = Jump(name="test", df=dataset)

In [200]:
jump.plt_exit_debug('km/u', 'm')

In [201]:
jump.plt_overview(['Elevation', 'Horizontal speed', 'Vertical speed'], 'km/u', 'm')

### Landing

A skydive swoop landing refers to a specialized landing technique used in the sport of skydiving, particularly in the discipline of canopy piloting. It involves a high-speed approach and a low-altitude maneuver to rapidly decelerate and touch down smoothly.

During a skydive swoop landing, the skydiver, equipped with a high-performance parachute called a "swoop canopy," utilizes the canopy's aerodynamic properties to generate lift and control their descent. As the skydiver nears the landing area, they enter a steep dive, typically at a 45-degree angle or steeper, to build up speed.

While in the dive, the skydiver maintains precise control over the canopy's flight path using various inputs, such as toggles and weight shifts, to adjust the angle of attack and manage the speed. As they approach the ground, the skydiver gradually levels out the canopy and begins to flare, which involves rapidly reducing the forward speed and increasing lift.

The objective of a skydive swoop landing is to achieve maximum speed and maintain control throughout the approach, while timing the flare to touch down smoothly and safely. Skilled canopy pilots can execute swoop landings with impressive precision, covering considerable distances horizontally before coming to a controlled stop.

It is important to note that skydive swoop landings require advanced skills, experience, and specialized training. They are typically performed by experienced skydivers who have undergone specific canopy piloting instruction and have demonstrated proficiency in handling high-performance canopies. Safety precautions, such as proper equipment, thorough knowledge of landing areas, and adherence to established guidelines, are crucial when attempting swoop landings to mitigate the associated risks.

Thanks to the flysight we can visualize our swoop and get the following information out of it:
- **initiated turn:** This indicates the initiation altitude of the speed building maneuver. This may mean different things depending on the pilot's flying style. It is typically the point where the pilot released his brakes and began applying double front riser input.
- **max vertical speed:** This is the highest vertical speed during the turn (it usually occurs right before the rollout). It is a good indicator of how much power was built in the turn (wind has little effect on this metric).
- **started rollout:** This is the altitude where the rollout begins. It is defined as the point where vertical speed peaks and then monotonically decreases until ground level. You may still be turning during the rollout but you are not generating vertical speed. Usually only minor heading corrections occur during the rollout.
- **max horizontal speed:** This is the highest horizontal speed (combined speed in the x-axis and y-axis) during the turn and rollout. Wind will strongly influence this value (it will be higher during downwind swoops and lower during upwind swoops).
- **degrees of rotation:** This indicates the size of the turn (the total heading change during the speed building maneuver).
- **time to execute turn:** This is the elapsed time between the start of the turn and the beginning of the rollout.
- **time during rollout:** This is the elapsed time between the start of the rollout and the point where the pilot passes the entry gate. If the entry gate is missed no value will be computed.
- **time aloft during swoop:** This is the elapsed time between the entry gate and touchdown.
- **entry gate speed:** This is the x-axis speed when the pilot eclipses the entry gate. It is interpolated between the nearest points before and after the gate.
- **distance to stop:** This is the x-axis distance between the entry gate and the point where motion stops (it includes the slide or runout).
- **speed carve time:** This is the time elapsed during a speed run through an IPC speed carve course. It is calculated only if the heuristics detect a speed carve run.

In [202]:
class Landing:
    def __init__(self, df):
        self.start_elevation = 2000
        self.exit_df = Exit(df).get_exit_df()
        self.start_elevation_df = self.exit_df[self.exit_df.elevation <= self.start_elevation].reset_index(drop=True)

        if self.is_init_turn_detected():
            self.landing_df = self.start_elevation_df.iloc[self.get_init_turn():].reset_index(drop=True) 
        else:
            logging.info("No high performance landing detected")

    def get_init_turn(self):
        landing_vert_speed = find_peaks_lows(self.start_elevation_df['vert_speed_km/u'], thres_peaks=0.5, min_dist_peaks=0.1, thres_lows=0.75, min_dist_lows=5)        
        # The initiated turn is where the vertspeed is the lowest and starts to increase
        lows_keys = [low for low in landing_vert_speed['lows'] if low <= landing_vert_speed['peaks'][-1]]
        lows_values = [self.start_elevation_df['vert_speed_km/u'][low] for low in lows_keys]
        if lows_values:
            return lows_keys[lows_values.index(min(lows_values))]
        return None
    
    def is_init_turn_detected(self):
        return self.get_init_turn() is not None

    def get_max_horz_speed_id(self):
        return self.landing_df.horz_speed_mph.idxmax() if hasattr(self, 'landing_df') else None

    def get_max_vert_speed_id(self):
        return self.landing_df.vert_speed_mph.idxmax() if hasattr(self, 'landing_df') else None

    def get_stop_estimate(self):
        offset = 1.5
        landing_horz_speed = find_peaks_lows(self.landing_df['horz_speed_km/u'], thres_peaks=0.1, min_dist_peaks=1, thres_lows=0.5, min_dist_lows=1)
        return [l for l in landing_horz_speed['lows'] if l > landing_horz_speed['peaks'][-1] and self.landing_df['horz_speed_km/u'][l] < offset][0]

    def get_start_rollout_estimate(self):
        landing_vert_speed = find_peaks_lows(self.landing_df['vert_speed_km/u'], thres_peaks=0.5, min_dist_peaks=0.1, thres_lows=0.75, min_dist_lows=0.1)
        last_peak_index = landing_vert_speed['peaks'][-1]

        diff = np.insert(np.diff(np.diff(self.landing_df['horz_distance_m'])), 0, 0)
        landing_horz_dis = pu.indexes(diff, thres=0.2, min_dist=0.1)

        for index in landing_horz_dis:
            if index >= last_peak_index:
                return index
        return None
    
    def get_stop_rollout_estimate(self):
        landing_vert_speed = find_peaks_lows(self.landing_df['vert_speed_km/u'], thres_peaks=0.5, min_dist_peaks=0.1, thres_lows=0.75, min_dist_lows=0.1)
        peaks = landing_vert_speed['peaks']
        lows = landing_vert_speed['lows']

        if len(peaks) > 0:
            first_low_after_last_peak = next((low for low in lows if low > peaks[-1]), None)
            if first_low_after_last_peak is not None:
                return first_low_after_last_peak
        return None
        
    def get_time_to_execute_turn(self):
        return f"{round(self.landing_df.time[self.get_start_rollout_estimate()] - self.landing_df.time[0], 1)} sec"
    
    def get_time_during_rollout(self):
        return f"{round(self.landing_df.time[self.get_stop_rollout_estimate()] - self.landing_df.time[self.get_start_rollout_estimate()], 1)} sec"
    
    def get_time_aloft_during_swoop(self):
        return f"{round(self.landing_df.time[self.get_stop_estimate()] - self.landing_df.time[self.get_stop_rollout_estimate()], 1)} sec"
    
    def set_start_point(self):
        key = self.get_stop_rollout_estimate()
        if key < 0 or key >= len(self.landing_df):
            raise ValueError("Invalid key for setting the starting point")
        self.landing_df = set_start_point(self.landing_df, key)
    
    def get_landing_df(self):
        return self.landing_df if hasattr(self, 'landing_df') else None
    
    def get_landing_data(self, speed_metric, distance_metric):
        x_axis = get_x_axis_settings(self.landing_df, distance_metric=distance_metric)
        y_axis = get_y_axis_settings(self.landing_df, speed_metric=speed_metric, distance_metric=distance_metric)
        x_dist, y_dist = get_x_axis_settings(self.landing_df, distance_metric='ft'), get_y_axis_settings(self.landing_df, distance_metric='ft')
        
        return f"""
        exited airplane:      {round(self.exit_df.elevation[0], 1)} ft AGL
        initiated turn:       {round(y_axis['Elevation']['col'][0], 1)} {y_axis['Elevation']['metric']} AGL, {abs(round(x_dist['Distance']['col'][0], 1))} {x_dist['Distance']['metric']} back, {-abs(round(y_dist['Distance']['col'][0], 1))} {y_dist['Distance']['metric']} offset
        max vertical speed:   {round(y_axis['Elevation']['col'][self.get_max_vert_speed_id()], 1)} {y_axis['Elevation']['metric']} AGL, {abs(round(x_dist['Distance']['col'][self.get_max_vert_speed_id()], 1))} {x_dist['Distance']['metric']} back, {-abs(round(y_dist['Distance']['col'][self.get_max_vert_speed_id()], 1))} {y_dist['Distance']['metric']} offset ({round(y_axis['Vertical speed']['col'][self.get_max_vert_speed_id()], 1)} {y_axis['Vertical speed']['metric']})
        started rollout:      {round(y_axis['Elevation']['col'][self.get_start_rollout_estimate()], 1)} {y_axis['Elevation']['metric']} AGL, {abs(round(x_dist['Distance']['col'][self.get_start_rollout_estimate()], 1))} {x_dist['Distance']['metric']} back, {-abs(round(y_dist['Distance']['col'][self.get_start_rollout_estimate()], 1))} {y_dist['Distance']['metric']} offset ({round(y_axis['Vertical speed']['col'][self.get_start_rollout_estimate()], 1)} {y_axis['Vertical speed']['metric']})
        finished rollout:     {round(y_axis['Elevation']['col'][self.get_stop_rollout_estimate()], 1)} {y_axis['Elevation']['metric']} AGL, {abs(round(x_dist['Distance']['col'][self.get_stop_rollout_estimate()], 1))} {x_dist['Distance']['metric']} back, {round(y_dist['Distance']['col'][self.get_stop_rollout_estimate()], 1)} {y_dist['Distance']['metric']} offset
        max horizontal speed: {round(y_axis['Elevation']['col'][self.get_max_horz_speed_id()], 1)} {y_axis['Elevation']['metric']} AGL, {abs(round(x_dist['Distance']['col'][self.get_max_horz_speed_id()], 1))} {x_dist['Distance']['metric']} back, {-abs(round(y_dist['Distance']['col'][self.get_max_horz_speed_id()], 1))} {y_dist['Distance']['metric']} offset ({round(y_axis['Horizontal speed']['col'][self.get_max_horz_speed_id()], 1)} {y_axis['Horizontal speed']['metric']})

        degrees of rotation:      ---- deg (--- -hand)
        time to execute turn:     {self.get_time_to_execute_turn()}
        time during rollout:      {self.get_time_during_rollout()}
        time aloft during swoop:  {self.get_time_aloft_during_swoop()}

        entry gate speed:      {round(y_axis['Horizontal speed']['col'][self.get_stop_rollout_estimate()])} {y_axis['Horizontal speed']['metric']}
        distance to stop:      {round(x_axis['Horizontal Distance']['col'][self.get_stop_estimate()])} {x_axis['Horizontal Distance']['metric']}
        """
    
    def plt_init_turn_debug(self, speed_metric, distance_metric):        
        x_axis = get_x_axis_settings(self.start_elevation_df, distance_metric=distance_metric)
        y_axis = get_y_axis_settings(self.start_elevation_df, speed_metric=speed_metric)
        
        landing_vert_speed = find_peaks_lows(y_axis['Vertical speed']['col'], thres_peaks=0.5, min_dist_peaks=0.1, thres_lows=0.75, min_dist_lows=5)
            
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=x_axis['Horizontal Distance']['col'], 
            y=y_axis['Vertical speed']['col'],
            line=dict(color=y_axis['Vertical speed']['color'], width=1.2),
            hovertemplate=y_axis['Vertical speed']['hovertemplate'], 
            showlegend=False
        ))
        fig.add_trace(go.Scatter(
            mode='markers', 
            x=x_axis['Horizontal Distance']['col'][landing_vert_speed['peaks']], 
            y=y_axis['Vertical speed']['col'][landing_vert_speed['peaks']],
            marker=dict(size=7, color='Green'),
            showlegend=False
        ))
        fig.add_trace(go.Scatter(
            mode='markers', 
            x=x_axis['Horizontal Distance']['col'][landing_vert_speed['lows']], 
            y=y_axis['Vertical speed']['col'][landing_vert_speed['lows']],
            marker=dict(size=7, color='Red'),
            showlegend=False
        ))
        
        fig.update_xaxes(title=f"Horizontal Distance ({x_axis['Horizontal Distance']['metric']})")
        fig.update_yaxes(title=f"Vertical speed ({y_axis['Vertical speed']['metric']})")
        
        if self.is_init_turn_detected():
            fig.add_vline(x=x_axis['Horizontal Distance']['col'][self.get_init_turn()], line_width=1.2)
        fig.update_layout(hovermode='x unified', showlegend=False, title_text="Initiated turn - Debug")
        return fig
    
    def plt_stop_estimate_debug(self):
        if hasattr(self, 'landing_df'):
            x_axis = get_x_axis_settings(self.landing_df)
            y_axis = get_y_axis_settings(self.landing_df)
            
            landing_horz_speed = find_peaks_lows(y_axis['Horizontal speed']['col'], thres_peaks=0.5, min_dist_peaks=0.1, thres_lows=0.75, min_dist_lows=5)
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=x_axis['Horizontal Distance']['col'], 
                y=y_axis['Horizontal speed']['col'],
                line=dict(color=y_axis['Horizontal speed']['color'], width=1.2),
                hovertemplate=y_axis['Horizontal speed']['hovertemplate'], 
                showlegend=False
            ))
            fig.add_trace(go.Scatter(
                mode='markers', 
                x=x_axis['Horizontal Distance']['col'][landing_horz_speed['peaks']], 
                y=y_axis['Horizontal speed']['col'][landing_horz_speed['peaks']],
                marker=dict(size=7, color='Green'),
                showlegend=False
            ))
            fig.add_trace(go.Scatter(
                mode='markers', 
                x=x_axis['Horizontal Distance']['col'][landing_horz_speed['lows']], 
                y=y_axis['Horizontal speed']['col'][landing_horz_speed['lows']],
                marker=dict(size=7, color='Red'),
                showlegend=False
            ))
            fig.add_vline(x=x_axis['Horizontal Distance']['col'][self.get_stop_estimate()], line_width=1.2)
            fig.update_xaxes(title=f"Horizontal Distance ({x_axis['Horizontal Distance']['metric']})")
            fig.update_yaxes(title=f"Horizontal speed ({y_axis['Horizontal speed']['metric']})")
            fig.update_layout(hovermode='x unified',legend=dict(yanchor="top", y=1, xanchor="right", x=1), title_text="Stop Estimate - Debug")
            return fig
        
    def plt_rollout_debug(self):
        if hasattr(self, 'landing_df'):
            fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], vertical_spacing=0.1, shared_xaxes=True, shared_yaxes=False, subplot_titles=("Vertical speed", "Horizontal distance"))
            
            x_axis = get_x_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1])
            y_axis = get_y_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1])
            
            landing_vert_speed = find_peaks_lows(y_axis['Vertical speed']['col'], thres_peaks=0.5, min_dist_peaks=0.1, thres_lows=0.75, min_dist_lows=0.1)            
            
            diff = np.insert(np.diff(np.diff(x_axis['Horizontal Distance']['col'])), 0, 0)
            landing_horz_dis = pu.indexes(diff, thres=0.2, min_dist=0.1)

            fig.add_trace(
                go.Scatter(
                    x=x_axis['Time']['col'], 
                    y=y_axis['Vertical speed']['col'],
                    line=dict(color=y_axis['Vertical speed']['color'], width=1.2),
                    hovertemplate=y_axis['Vertical speed']['hovertemplate'], 
                    showlegend=False
                ),
                row=1, col=1
            )
            fig.add_trace(
                go.Scatter(
                    mode='markers', 
                    x=x_axis['Time']['col'][landing_vert_speed['peaks']], 
                    y=y_axis['Vertical speed']['col'][landing_vert_speed['peaks']],
                    marker=dict(size=7, color='Green'),
                    showlegend=False
                ),
                row=1, col=1
            )
            fig.add_trace(
                go.Scatter(
                    mode='markers', 
                    x=x_axis['Time']['col'][landing_vert_speed['lows']], 
                    y=y_axis['Vertical speed']['col'][landing_vert_speed['lows']],
                    marker=dict(size=7, color='Red'),
                    showlegend=False
                ),
                row=1, col=1
            )
            fig.add_vline(x=x_axis['Time']['col'][self.get_start_rollout_estimate()], line_width=1.2, row=1, col=1)
            fig.add_vline(x=x_axis['Time']['col'][self.get_stop_rollout_estimate()], line_width=1.2, row=1, col=1)
            fig.update_xaxes(title=f"Time ({x_axis['Time']['metric']})", row=1, col=1)
            fig.update_yaxes(title=f"Vertical speed ({y_axis['Vertical speed']['metric']})", row=1, col=1)
            
            fig.add_trace(
                go.Scatter(
                    x=x_axis['Time']['col'],
                    y=x_axis['Horizontal Distance']['col'],
                    name='Horizontal Distance',
                    line=dict(color=x_axis['Horizontal Distance']['color'], width=1.2),
                    hovertemplate=x_axis['Horizontal Distance']['hovertemplate'],
                    showlegend=False
                ), 
                row=1, col=2
            )
            fig.add_trace(
                go.Scatter(
                    mode='markers', 
                    x=x_axis['Time']['col'][landing_horz_dis], 
                    y=x_axis['Horizontal Distance']['col'][landing_horz_dis],
                    marker=dict(size=7, color='Orange'),
                    showlegend=False
                ),
                row=1, col=2
            )
            
            fig.add_vline(x=x_axis['Time']['col'][self.get_start_rollout_estimate()], line_width=1.2, row=1, col=2)
            fig.add_vline(x=x_axis['Time']['col'][self.get_stop_rollout_estimate()], line_width=1.2, row=1, col=2)
            fig.update_xaxes(title=f"Time ({x_axis['Time']['metric']})", row=1, col=2)
            fig.update_yaxes(title=f"Horizontal Distance ({x_axis['Horizontal Distance']['metric']})", row=1, col=2)
            
            fig.update_layout(hovermode='x unified', legend=dict(yanchor="top", y=1, xanchor="right", x=1), title_text="Rollout - Debug")
            return fig
 
    def plt_overview(self, selected, speed_metric, distance_metric):
        if hasattr(self, 'landing_df'):
            if len(selected) != 0:
                x_axis = get_x_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1], distance_metric=distance_metric)
                y_axis = get_y_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1], speed_metric=speed_metric)

                plotly_data = []
                layout_kwargs = {'xaxis': {'domain': [0.06 * (len(selected) - 1), 1],
                                           'title': 'Horizontal distance (' + distance_metric + ')'}}

                for i, s in enumerate(selected):
                    axis_name = 'yaxis' + str(i + 1) * (i > 0)
                    yaxis = 'y' + str(i + 1) * (i > 0)
                    plotly_data.append(go.Scatter(
                        x=x_axis['Horizontal Distance']['col'],
                        y=y_axis[s]['col'],
                        name=s,
                        mode='lines',
                        line=dict(color=y_axis[s]['color'], width=1.2),
                        showlegend=False,
                        hovertemplate=y_axis[s]['hovertemplate'],
                    ),
                    )
                    layout_kwargs[axis_name] = {
                        'position': i * 0.06, 'side': 'left', 'title': s + ' (' + y_axis[s]['metric'] + ')',
                        'titlefont': {'size': 10, 'color': y_axis[s]['color']}, 'title_standoff': 0,
                        'anchor': 'free', 'tickfont': {'size': 10, 'color': y_axis[s]['color']}, 'showgrid': False,
                    }

                    plotly_data[i]['yaxis'] = yaxis
                    if i > 0:
                        layout_kwargs[axis_name]['overlaying'] = 'y'

                fig = go.Figure(data=plotly_data, layout=go.Layout(**layout_kwargs))
                fig.add_vline(x=x_axis['Horizontal Distance']['col'][self.get_start_rollout_estimate()], line_width=1.2, row=1, col=1)
                fig.add_vline(x=x_axis['Horizontal Distance']['col'][self.get_stop_rollout_estimate()], line_width=1.2, row=1, col=1)
                fig.update_layout(
                    hovermode="x unified"
                )
                return fig
            else:
                return go.Figure(empty_layout("Please select Y-axis"))
        
    def plt_overhead(self, distance_metric):
        if hasattr(self, 'landing_df'):
            x_axis = get_x_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1], distance_metric=distance_metric)
            y_axis = get_y_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1], distance_metric=distance_metric)

            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=x_axis['Distance']['col'],
                y=y_axis['Distance']['col'],
                line=dict(color=y_axis['Distance']['color'], width=1.2),
                hovertemplate=y_axis['Distance']['hovertemplate'],
                showlegend=False,
            ))
            fig.update_layout(hovermode='x unified',
                              legend=dict(yanchor="top", y=1, xanchor="right", x=1), title='Overhead view of flight path')

            # Set the aspect ratio to be equal
            fig.update_layout(xaxis_scaleanchor="y", yaxis_scaleanchor="x")
            return fig
            
    def plt_side_view(self, distance_metric):
        if hasattr(self, 'landing_df'):
            x_axis = get_x_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1], distance_metric=distance_metric)
            y_axis = get_y_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1])

            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=x_axis['Horizontal Distance']['col'],
                y=y_axis['Elevation']['col'],
                name='Elevation',
                line=dict(color=y_axis['Elevation']['color'], width=1.2),
                hovertemplate=y_axis['Elevation']['hovertemplate'],
                showlegend=False
            ))
            fig.update_layout(hovermode='x unified',
                              legend=dict(yanchor="top", y=1, xanchor="right", x=1), title='Side view of flight path')
            fig.update_xaxes(title=f"Horizontal distance ({x_axis['Horizontal Distance']['metric']})")
            fig.update_yaxes(title=f"Elevation ({y_axis['Elevation']['metric']})")

            return fig
        
    def plt_speed(self, speed_metric, distance_metric):
        if hasattr(self, 'landing_df'):
            x_axis = get_x_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1], distance_metric=distance_metric)
            y_axis = get_y_axis_settings(self.landing_df.iloc[:self.get_stop_estimate()+1], speed_metric=speed_metric)

            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=x_axis['Horizontal Distance']['col'],
                y=y_axis['Horizontal speed']['col'],
                line=dict(color=y_axis['Horizontal speed']['color'], width=1.2),
                hovertemplate=y_axis['Horizontal speed']['hovertemplate'],
                showlegend=False
            ))
            fig.add_trace(go.Scatter(
                x=[x_axis['Horizontal Distance']['col'].iloc[self.get_max_horz_speed_id()]],
                y=[y_axis['Horizontal speed']['col'][self.get_max_horz_speed_id()]],
                name=f"max horz speed ({round(y_axis['Horizontal speed']['col'][self.get_max_horz_speed_id()], 2)} {y_axis['Horizontal speed']['metric']})",
                text=["Max Horz Speed"],
                textposition="top right",
                hovertemplate='max horz speed: %{y:.2f}' + y_axis['Horizontal speed']['metric'] + ' <extra></extra>',
                mode='markers+text'
            ))

            fig.update_layout(hovermode='x unified',
                              legend=dict(yanchor="top", y=1, xanchor="right", x=1))
            fig.update_xaxes(title=f"Horizontal distance ({x_axis['Horizontal Distance']['metric']})")
            fig.update_yaxes(title=f"Horizontal speed ({y_axis['Horizontal speed']['metric']})")
            return fig
        
    def plt_map(self):
        if hasattr(self, 'landing_df'):
            y_axis = get_y_axis_settings(self.landing_df)

            fig = go.Figure()

            fig.add_trace(go.Scattermapbox(
                lat=self.landing_df.lat.iloc[:self.get_stop_estimate()+1],
                lon=self.landing_df.lon.iloc[:self.get_stop_estimate()+1],
                mode='lines',
                line=dict(color=y_axis['Elevation']['color']),
                showlegend=False,
            ))
            fig.update_layout(
                margin=dict(l=0, r=0, t=0, b=0),
                legend=dict(yanchor="top", y=1, xanchor="right", x=1),
                mapbox=dict(
                    accesstoken=token,
                    style="satellite-streets",
                    center=go.layout.mapbox.Center(
                        lat=self.landing_df.lat[round(len(self.landing_df) / 2)],
                        lon=self.landing_df.lon[round(len(self.landing_df) / 2)]
                    ),
                    pitch=0,
                    zoom=15.5
                )
            )
            return fig
        
        
class Jump(Landing, Exit):
    def __init__(self, name="empty", df=pd.DataFrame()):
        self.name = name
        if not df.empty:
            self.df = df
            super().__init__(self.df)

    def get_df(self):
        return self.df

    def get_name(self):
        return self.name

    def set_name(self, name):
        self.name = name

    def __str__(self):
        return f'{self.name}'

In [203]:
class Jump(Landing, Exit):
    def __init__(self, name="empty", df=pd.DataFrame()):
        self.name = name
        if not df.empty:
            self.df = df
            super().__init__(self.df)

    def get_df(self):
        return self.df

    def get_name(self):
        return self.name

    def set_name(self, name):
        self.name = name

    def __str__(self):
        return f'{self.name}'

In [204]:
landing = Jump(name="test", df=dataset)
landing.set_start_point()

#### Debugging Visualisations


<br>


**Initiated Turn**

This indicates the initiation altitude of the speed building maneuver.This may mean different things depending on the pilot's flying style.It is typically the point where the pilot released his brakes and began applying double front riser input.

**The initiated turn is where the vertspeed is the lowest and starts to increase**

In [205]:
landing.plt_init_turn_debug('km/u', 'm')

**Stop**


This signifies the conclusion of the landing, marking the cessation of motion.

The stop estimation is determined by identifying the initial dip in horizontal speed data after the most recent peak, where the horizontal speed falls below a predefined offset value. It serves as a method to approximate the point at which the horizontal speed reaches a specific threshold subsequent to reaching its final peak.

In [206]:
landing.plt_stop_estimate_debug()

**Rollout**


This is the point at which the rollout initiates. It is characterized by the moment when the vertical speed reaches its highest point and subsequently decreases continuously until reaching ground level. While you may still be making turns during the rollout, you are no longer ascending or descending vertically. Typically, only minor adjustments to your heading are made during this phase.

To determine the beginning of the rollout, we identify the vertical speed peak. The rollout commences as the horizontal distance starts to increase.

The conclusion of the rollout occurs at the first instance of the lowest vertical point following the peak.

In [207]:
landing.plt_rollout_debug()

#### Metrics

In [208]:
print(textwrap.dedent(landing.get_landing_data('km/u', 'm')).strip())

exited airplane:      4915.8 ft AGL
initiated turn:       765.8 ft AGL, 80.5 ft back, -348.2 ft offset
max vertical speed:   255.4 ft AGL, 223.6 ft back, -84.1 ft offset (119.2 km/u)
started rollout:      191.3 ft AGL, 220.7 ft back, -87.5 ft offset (116.0 km/u)
finished rollout:     -2.1 ft AGL, 0.0 ft back, 0.0 ft offset
max horizontal speed: 433.7 ft AGL, 135.0 ft back, -64.9 ft offset (87.9 km/u)

degrees of rotation:      ---- deg (--- -hand)
time to execute turn:     9.8 sec
time during rollout:      4.0 sec
time aloft during swoop:  7.0 sec

entry gate speed:      65 km/u
distance to stop:      50 m


### Visualisations

<br>

Visual aids that simplify the process of identifying landings within the flysight data.

In [209]:
landing.plt_overview(['Elevation', 'Horizontal speed', 'Vertical speed'], 'km/u', 'm')

In [210]:
landing.plt_side_view('m')

In [211]:
landing.plt_overhead('ft')

In [212]:
landing.plt_map()

In [213]:
landing.plt_speed('km/u', 'm')

### Gates/Course

In competitive swooping, or canopy piloting, there are typically three distinct courses or gates that competitors must navigate with precision and skill. These courses are designed to showcase a canopy pilot's ability to control their parachute while performing various aerial maneuvers. Let's explore the three main courses or gates in a canopy piloting competition:

**Distance Course:** The distance course is all about speed and precision. Competitors start from a designated point and fly their parachutes low over the water, aiming to cover the longest horizontal distance before touching down. The goal is to fly as close to the water's surface as safely possible while minimizing altitude loss. The pilot must make a well-timed and accurate landing to maximize their score. This course tests a pilot's ability to balance speed and control.

**Accuracy Course:** The accuracy course challenges canopy pilots to showcase their pinpoint precision. Competitors are required to navigate through a series of gates or obstacles, which are typically set up over a body of water. The aim is to fly through these gates while maintaining perfect control and hitting a designated target, such as a small, floating disc. Points are awarded based on how close the pilot's canopy lands to the target. Precision and control are key factors in this course.

**Speed Course:** The speed course is all about aerial finesse. Competitors perform high-speed, low-altitude turns and maneuvers, often referred to as "swoops." The objective is to fly a specific course with the fastest time and make a skillful landing. Pilots use their skill and technique to maintain speed, control, and accuracy while navigating this challenging course. Judges evaluate factors like style and finesse in addition to time.